### Example of training neural network for multi hazard project

In [30]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from matplotlib import rc
rc('text', usetex=False)
import seaborn as sns
import pandas as pd
import numpy as np
import sklearn
#!pip install shap
import shap
from sklearn.preprocessing import MinMaxScaler
#!pip uninstall keras-nightly
from tensorflow import keras
import re
from bayes_opt import BayesianOptimization
import pickle
import tensorflow as tf

In [31]:
# Moved all helper functions (such as plot functions) to utils/utils.py
# Here we import them
from utils.utils import (R_squared, plot_2_metric_mean)

In [32]:
# Moved all model functions to utils/model.py
from utils.model import (my_model0, my_model)

### Load the data from csv file

In [33]:
df = pd.read_csv('data/v27.csv')

In [34]:
X = df[['int_drought',
       'int_earthquake', 'int_extreme_temp',
       'int_flood', 'int_landslide',
       'int_tropical', 'int_unknown_storm', 'int_conv_storm',
        # background variables
        'mm_fault_density', 'mm_slope',
          'mm_road_density',
        # vulnerability variables
        'multi_phdi', 'multi_ppp','single_phdi', 'single_ppp']]
y = df['ln10_Total_Damage']

In [35]:
#split the data 80% training and 20% testin
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, random_state = 4)

### YS: Here the model is not defined yet -- please fix

In [13]:
X_train_summary = shap.kmeans(X_train, 10)
ex = shap.KernelExplainer(model.predict, X_train_summary)
#%%
# Dimension: last layer neuron # * sample # * feature #
shap_values = ex.shap_values(X_test.iloc[0:50,:])
# average over all neurons in the last layer

s = np.mean(shap_values, axis = 0)
shap.summary_plot(s, X_test.iloc[0:50,:])

#%% interaction term
plt.rcParams.update({'font.size': 25})
shap.dependence_plot("multi_phdi", s, X_test.iloc[0:50,:], interaction_index="mm_slope", show=False,cmap=plt.get_cmap("bwr"),dot_size=40)
plt.savefig('figures/trial_shap2.png', format='png', dpi=150, bbox_inches='tight')
#%%
# Save and plot mean of 3 runs

NameError: name 'model' is not defined

### YS: This bit runs -- I'll test it. 

In [9]:
#
# run my_model
#%%
learning_rate = 2e-3
regularization = 6
layer = 1
num_run = 1
history = [[] for i in range(num_run)]
val_metric = 0
for run_idx  in range(num_run):
    print('Same setting run: {}'.format(run_idx + 1))
    history_run = my_model0(learning_rate, regularization,layer, run_idx, X_train, y_train)
    history[run_idx] = history_run
    val_metric += history_run['val_loss'][-1]
p1 = str(learning_rate).replace(".", "_" )# learning rate
# hyperparameter 2
p2 = str(regularization).replace(".", "_")# regularization
p3 = round(layer) # activation function
with open('figures/adagrad_lr{}_reg{}_lay{}'.format(p1[:5],p2[:5],p3), "wb") as fp:   #Pickling
    pickle.dump(history, fp)

Same setting run: 1
1
Epoch 1/20
 58/213 [=======>......................] - ETA: 3s - loss: 416.7552 - R_squared: -0.0087

KeyboardInterrupt: 

In [14]:
#%% Bayes Optimization
pbounds = {'learning_rate': (1e-3, 5e-2),
           'regularization': (10,30),
           'layer':(-0.5, 2.5),
           #'drop_out_rate': (0.2, 0.5),
           #'momentum': (0.1, 0.5)
           }
optimizer = BayesianOptimization(
    f = my_model,
    pbounds = pbounds,
    random_state = 42)

optimizer.maximize(
    init_points = 1,
    n_iter = 4)
#%%
for i, res in enumerate(optimizer.res):
    print('Iteration {}: \n\t{}'.format(i, res))
    break

#%%

#file_name = "adagrad_lr1_985_reg41_44_lay1"#1.48/1.39/crazy r2
#file_name = "adagrad_lr0_061_reg5_541_lay1"#1.45/0.75/0.6/0.4
aaa = pickle.load(open( "figures/"+file_name, "rb"))
#%%
plt.plot(aaa[0]['loss'])
#%%
plt.plot(aaa[0]['val_loss'])
#%%
plt.plot(aaa[0]['R_squared'])
#%%
plt.plot(aaa[0]['val_R_squared'])


|   iter    |  target   |   layer   | learni... | regula... |
-------------------------------------------------------------


TypeError: my_model() missing 2 required positional arguments: 'X_train' and 'y_train'